# Módulo 2 - Preprocesado y Modelado

## Librerías

In [1]:
# === PyTorch y utilidades para deep learning ===
import torch  # Librería principal de PyTorch
from torchvision import datasets, transforms, models  # Módulos para datasets, transformaciones y modelos preentrenados
from torch.utils.data import DataLoader, random_split  # Utilidades para manejo de datos
import torch.nn as nn  # Módulo para definir arquitecturas de redes
import torch.optim as optim  # Optimizadores como SGD, Adam, etc.

# === Visualización ===
import matplotlib.pyplot as plt  # Visualización de gráficos y resultados
import seaborn as sns  # Visualización estadística

# === Utilidades adicionales ===
import kagglehub  # Descarga de datasets desde Kaggle Hub
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix  # Métricas de evaluación
import time  # Para medir tiempos de ejecución
import numpy as np  # Operaciones numéricas
from PIL import Image  # Manipulación de imágenes con Pillow


/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/RNABI2025-1-Equipo3/Trabajo3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

In [2]:
# Download latest version
images_path = kagglehub.dataset_download("arafatsahinafridi/multi-class-driver-behavior-image-dataset") + "/Multi-Class Driver Behavior Image Dataset"

print("Path to dataset files:", images_path)

Path to dataset files: /home/druiz35/.cache/kagglehub/datasets/arafatsahinafridi/multi-class-driver-behavior-image-dataset/versions/1/Multi-Class Driver Behavior Image Dataset


## Transformaciones

### Cambio de tamaño de imágenes

In [3]:
# Define las dimensiones de redimensionado que se usarán para las imágenes según el modelo seleccionado
resize_options = {
    "ResNet": (224, 224)  # Tamaño requerido por modelos tipo ResNet preentrenados
}

# Selección del tamaño de redimensionado (puede cambiarse según el modelo a utilizar)
resize_selection = resize_options["ResNet"]

# Transformación de redimensionado a aplicar a las imágenes
size_transformation = transforms.Resize(resize_selection)


### Normalización de valores de pixeles

In [4]:
# Define los parámetros de normalización según los valores utilizados en modelos preentrenados de PyTorch
normalization_options = {
    "TorchDefault": [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]  # Media y desviación estándar por canal (RGB)
}

# Selección del esquema de normalización (ajustable)
normalization_option = normalization_options["TorchDefault"]
norm_option_mean = normalization_option[0]
norm_option_std = normalization_option[1]

# Transformación de normalización a aplicar a las imágenes
norm_transformation = transforms.Normalize(mean=norm_option_mean, std=norm_option_std)


### Rotación aleatoria

In [5]:
# Transformación de aumento de datos: aplica una rotación aleatoria entre 0° y 180°
randRotation_transformation = transforms.RandomRotation(degrees=(0, 180))


### Rotación horizontal aleatoria 

In [6]:
# Transformación de aumento de datos: aplica una inversión horizontal con probabilidad del 50%
randHorizontalRotation_transformation = transforms.RandomHorizontalFlip(p=0.5)


### Transformación a Tensor

In [7]:
# Transformación que convierte una imagen PIL o NumPy array en un tensor de PyTorch y escala los valores a [0, 1]
toTensor_transformation = transforms.ToTensor()


### Concatenación de transformaciones

In [8]:
# Define y selecciona un pipeline de transformaciones a aplicar a las imágenes
pipeline_settings = {
    "Initial": [size_transformation, toTensor_transformation, norm_transformation]
}

# Selección del pipeline de transformación (puede modificarse para pruebas o entrenamiento)
pipeline_selection = pipeline_settings["Initial"]

# Composición final de las transformaciones a aplicar en secuencia
transform_pipeline = transforms.Compose([
    size_transformation,
    toTensor_transformation,
    norm_transformation
])


### Cargado y aplicación de las transformaciones

In [9]:
dataset = datasets.ImageFolder(images_path, transform=transform_pipeline)
class_to_idx = dataset.class_to_idx  # e.g., {'safe_driving': 0, 'texting_phone': 1, ...}
idx_to_class = {v: k for k, v in class_to_idx.items()}  # e.g., {0: 'safe_driving', ...}
print(idx_to_class)

['other_activities', 'safe_driving', 'talking_phone', 'texting_phone', 'turning']


## Train-Val-Test Split (70%, 20%, 10%)

In [10]:
# Divide el dataset en subconjuntos de entrenamiento (70%), validación (20%) y prueba (10%)
dataset_len = len(dataset)
train_len = int(0.7 * dataset_len)
val_len = int(0.2 * dataset_len)
test_len = dataset_len - train_len - val_len  # Asegura que la suma sea igual al total

train_set, val_set, test_set = random_split(dataset, [train_len, val_len, test_len])

# Crea los dataloaders para cada subconjunto, con batch size de 32
batch_size = 16
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True  # Mezcla aleatoriamente los datos en cada época (solo para entrenamiento)
)
val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False  # Sin mezcla para mantener consistencia en validación
)
test_loader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False  # Sin mezcla para evaluación final
)


## Función de pérdida y algoritmo de optimización

## Modelos

In [11]:
# Diccionario para ir guardando los modelos a probar
modelos = {}

### ResNet18

In [12]:
# Carga el modelo ResNet-18 preentrenado en ImageNet desde PyTorch Hub
resnet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)


Using cache found in /home/druiz35/.cache/torch/hub/pytorch_vision_v0.10.0
/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/RNABI2025-1-Equipo3/Trabajo3/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/RNABI2025-1-Equipo3/Trabajo3/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# Configura el modelo ResNet-18 dentro del dict de modelos con su optimizador y función de pérdida
modelos["Resnet18"] = {
    "model": resnet_model,
    "optimizer": optim.Adam(resnet_model.parameters(), lr=1e-4),  # Optimizador Adam con tasa de aprendizaje 1e-4
    "criterion": nn.CrossEntropyLoss()  # Función de pérdida para clasificación multiclase
}


### VGG19

In [14]:
# Carga el modelo VGG19 preentrenado y ajusta la última capa del clasificador para adaptarse a 5 clases
vgg19_model = models.vgg19(pretrained=True)
vgg19_model.classifier[6] = nn.Linear(4096, 5)  # Reemplaza la capa final para clasificación en 5 clases


/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/RNABI2025-1-Equipo3/Trabajo3/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# Configura el modelo VGG19 con su optimizador y función de pérdida, y lo agrega al diccionario de modelos
modelos["VGG19"] = {
    "model": vgg19_model,
    "optimizer": optim.Adam(vgg19_model.parameters(), lr=1e-4),  # Optimizador Adam con learning rate de 1e-4
    "criterion": nn.CrossEntropyLoss()  # Función de pérdida para clasificación multiclase
}


### Modelo custom

In [16]:
# Definición de una red neuronal convolucional personalizada (CNNAlpha) para clasificación en 6 clases
class CNNAlpha(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Reduce a la mitad las dimensiones espaciales
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Flatten(),
            nn.Linear(256 * 28 * 28, 1024),  # Asume entrada redimensionada a 224x224
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 5)  # Capa final con salida para 5 clases
        )
    
    def forward(self, xb):
        return self.network(xb)


In [17]:
# Instancia el modelo CNNAlpha y lo agrega al diccionario de modelos junto con su optimizador y función de pérdida
cnnalpha = CNNAlpha()
modelos["CNNAlpha"] = {
    "model": cnnalpha,
    "optimizer": optim.Adam(cnnalpha.parameters(), lr=1e-4),  # Optimizador Adam con tasa de aprendizaje 1e-4
    "criterion": nn.CrossEntropyLoss()  # Función de pérdida para clasificación multiclase
}


## Configuración de tracking de métricas

In [18]:
# === Función para guardar el modelo entrenado en disco ===
def save_model(model_name, model): 
    model_path = f"./{model_name}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model {model_name} saved to: {model_path}")

# === Función de entrenamiento y validación del modelo ===
def train_model(model, optimizer, device, criterion, model_name):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    num_epochs = 6  # Número de épocas de entrenamiento
    print("Starting training...")
    for epoch in range(num_epochs):
        print(f"\n Epoch {epoch + 1}/{num_epochs}")
        
        # --- Entrenamiento ---
        model.train()
        running_loss = 0.0
        correct_train = total_train = 0
        print("Entrenando con imágenes dadas...")
        counter = 1
        for images, labels in train_loader:
            print(f"Imagen #{counter}")
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
            counter += 1
        
        # Métricas de entrenamiento
        print("Calculando métricas...")
        train_acc = correct_train / total_train
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_acc)

        # --- Validación ---
        print("Iniciando validación...")
        model.eval()
        correct = total = 0
        val_loss = 0.0
        with torch.no_grad():
            counter = 1
            for images, labels in val_loader:
                print(f"Imagen #{counter}")
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                counter += 1

        # Métricas de validación
        val_acc = correct / total
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)

        print(f" Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
        print(f" Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc*100:.2f}%\n")
    
    # Guarda el modelo entrenado
    save_model(model_name, model)
    return train_losses, val_losses, train_accuracies, val_accuracies, model

# === Evaluación final en el set de prueba ===
def test_model(model, device, model_name):
    correct = total = 0
    all_preds = []
    all_labels = []
    all_probs = []
    inference_times = []

    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            start_time = time.time()
            outputs = model(images)
            end_time = time.time()

            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            inference_times.append(end_time - start_time)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu())

    # Metrics
    test_acc = accuracy_score(all_labels, all_preds)
    test_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    test_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    test_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    print(f"\nFinal Test Accuracy: {test_acc * 100:.2f}%")
    print(f"Precision (weighted): {test_precision:.4f}")
    print(f"Recall (weighted): {test_recall:.4f}")
    print(f"F1-score (weighted): {test_f1:.4f}")

    avg_infer_time = np.mean(inference_times)
    print(f"\nAverage Inference Time per Batch: {avg_infer_time:.4f}s")
    print(f"Average Inference Time per Sample: {avg_infer_time / batch_size:.6f}s")

    # Confusion Matrix
    class_names = [idx_to_class[i] for i in range(len(idx_to_class))]
    plot_confusion_matrix(all_labels, all_preds, class_names, model_name)


# === Visualización de la matriz de confusión ===
def plot_confusion_matrix(all_labels, all_preds, class_names, model_name):
    cm = confusion_matrix(all_labels, all_preds)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix on Test Set for {model_name}')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()

# === Visualización de los resultados de entrenamiento ===
def plot_results(train_accuracies, val_accuracies, train_losses, val_losses, num_epochs=10):
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(14, 5))

    # Gráfico de precisión
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_accuracies, 'o-', label='Train Accuracy')
    plt.plot(epochs, val_accuracies, 'o-', label='Val Accuracy')
    plt.title('Accuracy vs Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Gráfico de pérdida
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_losses, 'o-', label='Train Loss')
    plt.plot(epochs, val_losses, 'o-', label='Val Loss')
    plt.title('Loss vs Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


## Entrenamiento, Validación y Testing

In [ ]:
#del modelos["Resnet18"] # HOTFIX!!! COMENTAR SI NO SE HA ENTRENADO ESTE MODELO. 
#del modelos["VGG19"]

# Entrena y evalúa cada modelo definido en el diccionario 'modelos'
"""
for model_name, model_options in modelos.items():
    print(f"ENTRENAMIENTO DE MODELO: {model_name}")
    
    # Extrae modelo, optimizador y función de pérdida
    model = model_options["model"]
    optimizer = model_options["optimizer"]
    criterion = model_options["criterion"]
    device = "cpu"  # Se puede cambiar a "cuda" si hay GPU disponible

    # Entrenamiento del modelo
    train_losses, val_losses, train_accuracies, val_accuracies, model = train_model(
        model,
        optimizer,
        device,
        criterion,
        model_name
    )

    # Evaluación final sobre el conjunto de prueba
    test_model(model, device, model_name)

    #plot_results(train_accuracies, val_accuracies, train_losses, val_losses, num_epochs=10)
"""

ENTRENAMIENTO DE MODELO: CNNAlpha
Starting training...

 Epoch 1/3
Entrenando con imágenes dadas...
Imagen #1
Imagen #2
Imagen #3
Imagen #4
Imagen #5
Imagen #6
Imagen #7
Imagen #8
Imagen #9
Imagen #10
Imagen #11
Imagen #12
Imagen #13
Imagen #14
Imagen #15
Imagen #16
Imagen #17
Imagen #18
Imagen #19
Imagen #20
Imagen #21
Imagen #22
Imagen #23
Imagen #24
Imagen #25
Imagen #26
Imagen #27
Imagen #28
Imagen #29
Imagen #30
Imagen #31
Imagen #32
Imagen #33
Imagen #34
Imagen #35
Imagen #36
Imagen #37
Imagen #38
Imagen #39
Imagen #40
Imagen #41
Imagen #42
Imagen #43
Imagen #44


## Prueba

In [ ]:
# Carga modelos
resnet_path = "./Resnet18.pth"
vgg_path = "./VGG19.pth"
cnnalpha_path = "CNNAlpha.pth" 

modelos = {}
resnet = models.resnet18()
resnet.fc = nn.Linear(resnet.fc.in_features, 5)
resnet.load_state_dict(torch.load(resnet_path))
resnet.eval()
modelos["ResNet18"] = resnet

vgg = models.vgg19()
vgg.classifier[6] = nn.Linear(4096, 5)
vgg.load_state_dict(torch.load(vgg_path))
vgg.eval()
modelos["VGG19"] = vgg

cnnalpha = CNNAlpha()
cnnalpha.load_state_dict(torch.load(cnnalpha_path))
cnnalpha.eval()
modelos["CNNAlpha"] = cnnalpha


In [ ]:

for model_name, model in modelos.items():
    print(f"TESTING DE MODELO: {model_name}")
    
    device = "cpu"  # Se puede cambiar a "cuda" si hay GPU disponible
    # Evaluación final sobre el conjunto de prueba
    test_model(model, device, model_name)

## Conclusiones generales